# Estudio de las métricas utilizadas para Naive Bayes

En esta hoja se va a estudiar como de necesarias son las métricas calculadas para el entreno del modelo de Naive Bayes.

Para realizar el estudio vamos a quitar del dataset los atributos relacionados con agrupamientos, con comunidades, con núcleos, y con centralidad, de esta forma veremos como incide en los resultados cada métrica.

Vamos a empezar importando las librerias y funciones que utilizaremos más adelante.

In [156]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos los atributos que calculamos para la realización de este trabajo. Luego, separamos el dataset en atributos por los cuales Naive Bayes se va a entrenar. Finalmente, elegiremos el objetivo, que en este caso es un objetivo categórico binario.

In [157]:
# Lectura de los csv

# Tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

# Tabla con todos los atributos sin los de agrupamiento
nodesWithoutClustering = pd.read_csv("../tablas/tableWithoutClustering.csv")

# Tabla con todos los atributos sin los de comunidades
nodesWithoutCommunity = pd.read_csv("../tablas/tableWithoutCommunity.csv")

# Tabla con todos los atributos sin los de nucleos
nodesWithoutKernel = pd.read_csv("../tablas/tableWithoutKernel.csv")

# Tabla con todos los atributos sin los de centralidad
nodesWithoutCentrality = pd.read_csv("../tablas/tableWithoutCentrality.csv")

In [158]:
# Preparación de los datos para la tabla con todos los atributos
atributos_all_discretos = ['name']
atributos_all_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos_all = nodes.loc[:,['id_node'] + atributos_all_discretos + atributos_all_continuos]

# Preparación de los datos para la tabla sin los atributos sin los de agrupamiento
atributos_WithoutClustering_discretos = ['name']
atributos_WithoutClustering_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos_WithoutClustering = nodesWithoutClustering.loc[:,['id_node'] + atributos_WithoutClustering_discretos + atributos_WithoutClustering_continuos]

# Preparación de los datos para la tabla sin los atributos sin los de comunidades
atributos_WithoutCommunity_discretos = ['name']
atributos_WithoutCommunity_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','Core number']
atributos_WithoutCommunity = nodesWithoutCommunity.loc[:,['id_node'] + atributos_WithoutCommunity_discretos + atributos_WithoutCommunity_continuos]

# Preparación de los datos para la tabla sin los atributos sin los de núcleos
atributos_WithoutKernel_discretos = ['name']
atributos_WithoutKernel_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','asyn_lpa_communities']
atributos_WithoutKernel = nodesWithoutKernel.loc[:,['id_node'] + atributos_WithoutKernel_discretos + atributos_WithoutKernel_continuos]

# Preparación de los datos para la tabla sin los atributos sin los de centralidad
atributos_WithoutCentrality_discretos = ['name']
atributos_WithoutCentrality_continuos = ['clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos_WithoutCentrality = nodesWithoutCentrality.loc[:,['id_node'] + atributos_WithoutCentrality_discretos + atributos_WithoutCentrality_continuos]


In [159]:
# Elección del objetivo
objetivo = nodes['ml_target']
objetivo.head()

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

# Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [160]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos_all[atributos_all_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación
codificador_atributos_discretos.fit(atributos_WithoutClustering[atributos_WithoutClustering_discretos]) 
codificador_atributos_discretos.fit(atributos_WithoutCommunity[atributos_WithoutCommunity_discretos]) 
codificador_atributos_discretos.fit(atributos_WithoutKernel[atributos_WithoutKernel_discretos]) 
codificador_atributos_discretos.fit(atributos_WithoutCentrality[atributos_WithoutCentrality_discretos]) 


# Ahora aplicamos el método transform para codificar los datos
atributos_all[atributos_all_discretos] = codificador_atributos_discretos.transform(
    atributos_all[atributos_all_discretos]
)
atributos_WithoutClustering[atributos_WithoutClustering_discretos] = codificador_atributos_discretos.transform(
    atributos_WithoutClustering[atributos_WithoutClustering_discretos]
)
atributos_WithoutCommunity[atributos_WithoutCommunity_discretos] = codificador_atributos_discretos.transform(
    atributos_WithoutCommunity[atributos_WithoutCommunity_discretos]
)
atributos_WithoutKernel[atributos_WithoutKernel_discretos] = codificador_atributos_discretos.transform(
    atributos_WithoutKernel[atributos_WithoutKernel_discretos]
)
atributos_WithoutCentrality[atributos_WithoutCentrality_discretos] = codificador_atributos_discretos.transform(
    atributos_WithoutCentrality[atributos_WithoutCentrality_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos_all['name'] = normalizador.fit_transform(atributos_all[['name']])
atributos_WithoutClustering['name'] = normalizador.fit_transform(atributos_WithoutClustering[['name']])
atributos_WithoutCommunity['name'] = normalizador.fit_transform(atributos_WithoutCommunity[['name']])
atributos_WithoutKernel['name'] = normalizador.fit_transform(atributos_WithoutKernel[['name']])
atributos_WithoutCentrality['name'] = normalizador.fit_transform(atributos_WithoutCentrality[['name']])

In [161]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_all_discretizados = atributos_all.copy()
atributos_WithoutClustering_discretizados = atributos_WithoutClustering.copy()
atributos_WithoutCommunity_discretizados = atributos_WithoutCommunity.copy()
atributos_WithoutKernel_discretizados = atributos_WithoutKernel.copy()
atributos_WithoutCentrality_discretizados = atributos_WithoutCentrality.copy()


atributos_all_discretizados[atributos_all_continuos] = discretizador.fit_transform(
    atributos_all_discretizados[atributos_all_continuos]
)
atributos_WithoutClustering_discretizados[atributos_WithoutClustering_continuos] = discretizador.fit_transform(
    atributos_WithoutClustering_discretizados[atributos_WithoutClustering_continuos]
)
atributos_WithoutCommunity_discretizados[atributos_WithoutCommunity_continuos] = discretizador.fit_transform(
    atributos_WithoutCommunity_discretizados[atributos_WithoutCommunity_continuos]
)
atributos_WithoutKernel_discretizados[atributos_WithoutKernel_continuos] = discretizador.fit_transform(
    atributos_WithoutKernel_discretizados[atributos_WithoutKernel_continuos]
)
atributos_WithoutCentrality_discretizados[atributos_WithoutCentrality_continuos] = discretizador.fit_transform(
    atributos_WithoutCentrality_discretizados[atributos_WithoutCentrality_continuos]
)

# El id_node no lo discretizamos, al igual que el name


 # Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

### Con todas las métricas

In [162]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [163]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo_all = cross_validate(tubería_NB1,
                                               atributos_all_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo_all

{'fit_time': array([0.036937  , 0.04374313, 0.04089642, 0.0404067 , 0.03677773,
        0.03590965, 0.03890109, 0.03790307, 0.03538179, 0.03758812]),
 'score_time': array([0.00598621, 0.00897765, 0.00897646, 0.00698328, 0.00667167,
        0.00598454, 0.00698256, 0.00700283, 0.00598598, 0.01895261]),
 'test_score': array([0.94552929, 0.67659138, 0.68172485, 0.65195072, 0.68069815,
        0.66016427, 0.70020534, 0.66427105, 0.65297741, 0.94661191])}

In [164]:
resultados_validación_cruzadaConAlphaOptimo_all['test_score'].mean()

0.7260724362721616

### Sin las métricas de agrupamiento

In [165]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo_WithoutClustering = cross_validate(tubería_NB1,
                                               atributos_WithoutClustering_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo_WithoutClustering

{'fit_time': array([0.0309217 , 0.03590989, 0.02992368, 0.02992654, 0.02892971,
        0.02813029, 0.02740312, 0.02714729, 0.02596116, 0.02593851]),
 'score_time': array([0.00797963, 0.00598431, 0.00658417, 0.00598192, 0.00505805,
        0.00495815, 0.00601506, 0.00501132, 0.00598764, 0.00598478]),
 'test_score': array([0.96505653, 0.66940452, 0.66632444, 0.63552361, 0.6724846 ,
        0.65400411, 0.68788501, 0.66427105, 0.65400411, 0.97227926])}

In [166]:
resultados_validación_cruzadaConAlphaOptimo_WithoutClustering['test_score'].mean()

0.7241237224359557

### Sin las métricas de comunidad

In [167]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo_WithoutCommunity = cross_validate(tubería_NB1,
                                               atributos_WithoutCommunity_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo_WithoutCommunity

{'fit_time': array([0.03369498, 0.03490996, 0.03445506, 0.03360343, 0.03092122,
        0.03045654, 0.02865815, 0.03033304, 0.03496623, 0.03113961]),
 'score_time': array([0.00698256, 0.00698233, 0.00635672, 0.00698185, 0.00552011,
        0.0053072 , 0.006675  , 0.00646377, 0.00601387, 0.00656581]),
 'test_score': array([0.94450154, 0.59240246, 0.58213552, 0.58829569, 0.5862423 ,
        0.57494867, 0.63655031, 0.58829569, 0.57802875, 0.94353183])}

In [168]:
resultados_validación_cruzadaConAlphaOptimo_WithoutCommunity['test_score'].mean()

0.6614932753122817

### Sin las métricas de núcleo

In [169]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo_WithoutKernel = cross_validate(tubería_NB1,
                                               atributos_WithoutKernel_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo_WithoutKernel

{'fit_time': array([0.03378606, 0.02902746, 0.02803874, 0.03055549, 0.02995563,
        0.03088093, 0.0297184 , 0.02967143, 0.03043199, 0.03016949]),
 'score_time': array([0.006387  , 0.00498724, 0.0059855 , 0.00564575, 0.00498772,
        0.004987  , 0.0059855 , 0.00562525, 0.00598621, 0.00498796]),
 'test_score': array([0.95786228, 0.67043121, 0.66529774, 0.64373717, 0.67864476,
        0.66427105, 0.69712526, 0.6550308 , 0.65195072, 0.96201232])}

In [170]:
resultados_validación_cruzadaConAlphaOptimo_WithoutKernel['test_score'].mean()

0.7246363308297334

### Sin las métricas de centralidad

In [171]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo_WithoutCentrality = cross_validate(tubería_NB1,
                                               atributos_WithoutCentrality_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo_WithoutCentrality

{'fit_time': array([0.03745127, 0.02844119, 0.025038  , 0.02487874, 0.02547359,
        0.02523804, 0.02636719, 0.02698612, 0.02599764, 0.02844644]),
 'score_time': array([0.00698543, 0.0069766 , 0.00598526, 0.00498724, 0.005018  ,
        0.00601387, 0.0050106 , 0.00501585, 0.00610232, 0.00598526]),
 'test_score': array([0.97636177, 0.68172485, 0.67145791, 0.66427105, 0.68172485,
        0.67351129, 0.70123203, 0.66324435, 0.64784394, 0.97741273])}

In [172]:
resultados_validación_cruzadaConAlphaOptimo_WithoutCentrality['test_score'].mean()

0.7338784765675286

# Conclusiones del estudio

Como podemos ver el siguente recuadro, para nuestro problema las métricas de comunidad son muy importante, ya que bajan la confianza en un 6,45%, las métricas de centralidad no son necesarias ya que sin ella el modelo de Naive Bayes consigue una mejor confianza, en el caso de no tener la de centralidad sube en un 0,79%. Por último sin las métricas de agrupamiento y de núcleo la confianza es prácticamente la misma (aunque empeora un poco), esto quiere decir que aunque no las tengamos se puede llegar a tener una buena confianza.


In [173]:
print("Confianza del dataser con todas las métricas:",resultados_validación_cruzadaConAlphaOptimo_all['test_score'].mean())
print("Confianza del dataser sin las métricas de agrupamiento:",resultados_validación_cruzadaConAlphaOptimo_WithoutClustering['test_score'].mean())
print("Confianza del dataser sin las métricas de comunidad:",resultados_validación_cruzadaConAlphaOptimo_WithoutCommunity['test_score'].mean())
print("Confianza del dataser sin las métricas de núcleo:",resultados_validación_cruzadaConAlphaOptimo_WithoutKernel['test_score'].mean())
print("Confianza del dataser sin las métricas de centralidad:",resultados_validación_cruzadaConAlphaOptimo_WithoutCentrality['test_score'].mean())

Confianza del dataser con todas las métricas: 0.7260724362721616
Confianza del dataser sin las métricas de agrupamiento: 0.7241237224359557
Confianza del dataser sin las métricas de comunidad: 0.6614932753122817
Confianza del dataser sin las métricas de núcleo: 0.7246363308297334
Confianza del dataser sin las métricas de centralidad: 0.7338784765675286
